498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9

In [7]:
type Material =
| Rock
| MovingSand
| SettledSand
| Air

type Position = {
    y: int
    x: int
}

type Cave = Map<Position,Material>

module Parser =
    let interpolate (starting,ending) =
        let sorted a b = if a < b then a,b else b,a
        if starting.x = ending.x then
            let (a, b) = sorted starting.y ending.y
            seq {
                for y in [a .. b] do 
                    yield { starting with y = y }
            }
        else
            let (a, b)  = sorted starting.x ending.x
            seq {
                for x in [a .. b] do 
                    yield { starting with x = x }
            }

    let parse (line:string) =
        line.Split("->")
        |> Seq.map ( fun it -> 
            let coords = it.Split(",") |> Array.map int
            { x = coords.[0]; y = coords.[1] } )
        |> Seq.pairwise
        |> Seq.map interpolate
        |> Seq.concat
        |> Seq.map (fun it -> it, Material.Rock)

module Cave =
    let dimensions (cave:Cave) =
        let maxBy chooser = 
            cave 
            |> Map.keys
            |> List.ofSeq
            |> List.map chooser
            |> List.max
        maxBy (fun it -> it.x), maxBy (fun it -> it.y)

    let preview (cave:Cave) =
        let width, height = cave |> dimensions
        seq {
            for y in [0 .. height] do yield seq {
                for x in [0 .. width] do
                    let key = { x = x; y = y }
                    if cave |> Map.containsKey key  then 
                        yield
                            match cave |> Map.find key with 
                            | Rock -> "🪨"
                            | MovingSand -> "🟡"
                            | SettledSand -> "🟨"
                            | Air -> "⬛"
                    else
                        yield "⬛"
            }
        }
        |> Seq.map (Seq.skip 450 >> String.concat "")
        |> String.concat "\n"

module Sand =
    let create (cave:Cave) =
        cave
        |> Map.add { x = 500; y = 0 } Material.MovingSand

    let move (sand:Position) (cave:Cave) =
        let width, height = cave |> Cave.dimensions
        if ( sand.y >= height ) then
            cave |> Map.remove sand // Sand already at the bottom of the cave
        else

        let down = { sand with y = sand.y + 1 }
        if cave |> Map.containsKey down then
            let downLeft = { down with x = sand.x - 1 }
            if cave |> Map.containsKey downLeft then
                let downRight = { down with x = sand.x + 1 }
                if cave |> Map.containsKey downRight then
                    cave |> Map.change sand (fun it -> match it with Some _ -> Some (Material.SettledSand) | _ -> None)
                else 
                    cave |> Map.remove sand |> Map.add downRight Material.MovingSand
            else
                cave
                |> Map.remove sand 
                |> Map.add downLeft Material.MovingSand
        else
        cave
        |> Map.remove sand
        |> Map.add down Material.MovingSand

module Simulation =
    let movingSand (cave:Cave) =
        cave 
            |> Map.toSeq 
            |> Seq.filter (fun (_,mat) -> match mat with Material.MovingSand -> true | _ -> false )
            |> Seq.map fst
            |> Seq.sortDescending
    
    let settledSand (cave:Cave) =
        cave 
            |> Map.toSeq 
            |> Seq.filter (fun (_,mat) -> match mat with Material.SettledSand -> true | _ -> false )
            |> Seq.map fst

    let tick (cave:Cave) =
        cave 
        |> movingSand
        |> Seq.fold( fun c s -> c |> Sand.move s ) cave
        |> Sand.create

    let rec run ticks cave =
        if ticks = 0 then cave else
        run (ticks - 1) (tick cave)

    let rec solve aTick aCave =
        if aTick > 100 then aCave, aTick else

        let _, height = aCave |> Cave.dimensions
        let cave = aCave |> tick  
        let sand = cave |> movingSand |> Seq.head
        if sand.y >= height then 
            cave, aTick
        else
            solve (aTick + 1) cave


let ResolutionFolder = __SOURCE_DIRECTORY__
let cave = 
    File.ReadLines( ResolutionFolder + "/testcase14.txt")
    |> Seq.map Parser.parse 
    |> Seq.concat
    |> Map.ofSeq

let solution, ticks =
    cave 
    //|> Cave.dimensions
    //|> Simulation.run 100 
    |> Simulation.solve 0
    //|> Simulation.movingSand
    //|> Simulation.settledSand |> Seq.length

display ticks

solution |> Simulation.settledSand |> Seq.length |> display

solution |> Cave.preview

33

⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡🟨⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡🟨🟨🟨⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡🪨🟨🟨🟨🪨🪨
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡🟨🪨🟨🟨🟨🪨⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡🪨🪨🪨🟨🟨🟨🪨⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡⬛⬛🟨🟨🟨🟨🪨⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡🟨⬛🟨🟨🟨🟨🟨🪨⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛🟡🪨🪨🪨🪨🪨🪨🪨🪨🪨⬛